In [1]:
import numpy as np
import matplotlib.pyplot as plt
from chart_studio import plotly
import plotly.tools as tls         # plotly tools
import plotly.graph_objs as go     # plot and configuration tools : Scatter, Line, Layout
import plotly.express as px


In [2]:
# Open the file in read mode
with open('openmx.Band', 'r') as file:
    tot_kpoints = 0 
    high_sym = [1] # Get the position of High Symmetry point. Encounter the double count highsymm point issue (require -1). So set the initial highsymm = 1 

    # read the Fermi energy in first line
    first_line = file.readline().split()
    num_bands = int(first_line[1])
    fermi_in_hartree = float(first_line[2])
    # read the Lattice vector
    lattice_vector_str= file.readline().split()
    lattice_vector = np.array([float(i) for i in lattice_vector_str]).reshape(3,3)
    # read the line section
    num_lines = int(file.readline())
    aux = []
    for i in range(num_lines):
        current_line = file.readline().split()
        print(current_line)
        num_points = int(current_line[0])
        init_coord = np.array([float(i) for i in current_line[1:4]])
        final_coord = np.array([float(i) for i in current_line[4:7]])
        print(num_points)
        print(init_coord)
        points = np.zeros((num_points, 3)) 
        for j in range(num_points):
            aux.append((init_coord + j * (final_coord - init_coord) / (num_points - 1)))
            points[j] = (init_coord + j * (final_coord - init_coord) / (num_points - 1))
        tot_kpoints += num_points
        high_sym.append(tot_kpoints)  

['20', '0.3333333333333333', '0.6666666666666667', '0', '0', '0', '0', 'K', 'Γ']
20
[0.33333333 0.66666667 0.        ]
['20', '0', '0', '0', '0.5', '0.5', '0', 'Γ', 'M']
20
[0. 0. 0.]
['15', '0.5', '0.5', '0', '0.6666666666666667', '0.3333333333333333', '0', 'M', "K'"]
15
[0.5 0.5 0. ]


In [3]:
# Calculate the relative distance according to k points
aux = np.array(aux)
kx = aux[:,0]
ky = aux[:,1]
kz = aux[:,2]
kd = []
for i in range(len(aux)):
    if i ==0:
        kdr = 0
        kd.append(kdr)
    else:
        kdr = np.linalg.norm(aux[i] - aux[i-1])
        kd.append(kd[i-1] + kdr)

In [4]:
with open('openmx.Band', 'r') as file:
    # skip the header
    num_bands = 50
    for _ in range(6):
        next(file)
    lines = file.readlines()
    energy_per_kp = []
    i = 0
    while i < len(lines):
        if lines[i].split()[0] == '{}'.format(num_bands):  # Check if the line starts with a number
            num_points = int(lines[i].split()[0])
            print(num_points)
            # Read coordinates
            coords = np.array([float(x) for x in lines[i].strip().split()[1:]])
            i += 1
            print(coords)
            
            energies = np.array([float(x) for x in lines[i].strip().split()])
            i+=1
            print(energies)
            energy_per_kp.append(energies)
        else:
            i += 1


50
[0.33333333 0.66666667 0.        ]
[-0.13204389 -0.13212145 -0.13216951 -0.13197993 -0.13231795 -0.13306566
 -0.13316384 -0.13053202 -0.13022025 -0.1301003  -0.13457797 -0.13501066
 -0.12907839 -0.13511293 -0.12894556 -0.12884939 -0.13574877 -0.12705695
 -0.13869352 -0.12449279 -0.12438229 -0.14045296 -0.12369377 -0.14059752
 -0.12332992 -0.12305023 -0.12286454 -0.12275217 -0.12244193 -0.14177893
 -0.1222554  -0.12186041 -0.1218161  -0.12148158 -0.12137833 -0.14295606
 -0.12114486 -0.12111338 -0.121087   -0.12092698 -0.14342527 -0.12066475
 -0.12037786 -0.12019511 -0.1200525  -0.11994182 -0.11986431 -0.11967349
 -0.11944313 -0.11928723]
50
[0.31578947 0.63157895 0.        ]
[-0.13204662 -0.13200551 -0.1321809  -0.13229456 -0.13182933 -0.13309455
 -0.13315849 -0.13067093 -0.13035785 -0.13007991 -0.13460474 -0.13499608
 -0.13505217 -0.12908927 -0.12894701 -0.12875606 -0.13575336 -0.12719599
 -0.13843025 -0.12465125 -0.12458223 -0.14014903 -0.12387977 -0.12343533
 -0.12317012 -0.141105

In [5]:
# Create a list to store all scatter traces
traces = []
hartree_to_ev = 27.211407953
fermi = fermi_in_hartree * hartree_to_ev

# Iterate through kd and energy_per_kp to create scatter traces
for xe, ye in zip(kd, energy_per_kp):
    shift_w_fermi = np.sort(ye * hartree_to_ev - fermi)
    traces.append(go.Scatter(
        x=[xe] * len(ye),
        y=shift_w_fermi,
        mode='markers',
        marker=dict(color='blue'),
    ))

# Create the layout
layout = go.Layout(
    title='Energy vs kd',
    xaxis=dict(title='kd'),
    yaxis=dict(title='Energy per kp', range=[-0.5, 0.5]) 
)




In [6]:
import plotly.graph_objects as go

# Assuming traces and tot_kpoints are already defined
emin = -0.5
emax = 0.5
labels = [r"$K$", r"$\Gamma$", r"$M$", r"$K$"]
step = max(kd) / (len(labels) - 1)
# Create vertical lines at high-symmetry points
vlines = []
for i, label in enumerate(labels):
    vlines.append(
        go.Scatter(
            x=[kd[high_sym[i]-1], kd[high_sym[i]-1]],
            y=[emin, emax],
            mode="lines",
            line=dict(color="#111111", width=1),
            showlegend=False
        )
    )

# Add annotations for the high-symmetry points
annotations = []
for i, label in enumerate(labels):
    annotations.append(
        dict(
            x=kd[high_sym[i]-1], y=emin,
            xref="x", yref="y",
            text=label,
            xanchor="center", yanchor="top",
            showarrow=False
        )
    )

# Define the x and y axis layout
bandxaxis = dict(
    title="k-points",
    range=[0, max(kd)],
    showgrid=True,
    showline=True,
    ticks="",
    showticklabels=False,
    mirror=True,
    linewidth=2
)
bandyaxis = dict(
    title=r"$E - E_f \quad / \quad \text{eV}$",
    range=[emin, emax],
    showgrid=True,
    showline=True,
    zeroline=True,
    mirror="ticks",
    ticks="inside",
    linewidth=2,
    tickwidth=2,
    zerolinewidth=2
)

# Add a horizontal line at y=0 to represent the Fermi energy
fermi_line = go.Scatter(
    x=[0, max(kd)],  # From the start to the end of the x-axis
    y=[0, 0],        # y = 0 for both points
    mode="lines",
    line=dict(color="red", width=2),  # Red line
    showlegend=False
)

# Define the layout for the plot
bandlayout = go.Layout(
    title="Bands diagram of TBB",
    xaxis=bandxaxis,
    yaxis=bandyaxis,
    annotations=annotations,
    showlegend=False
)

# Combine traces and vertical lines into the figure data
fig = go.Figure(data=traces + vlines + [fermi_line], layout=bandlayout)

# Display the plot (for Jupyter Notebooks)
# fig.show()

# Alternatively, to save the plot as a file:
import plotly.io as pio
pio.write_html(fig, file='TBB.html', auto_open=True)

# If you want to embed the plot in an HTML page:
# plot_url = pio.to_html(fig, include_plotlyjs='cdn')
# print(plot_url)
